# Loading Dataset

In [ ]:
!ls

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# !rm -r sample_data
# !cp -r drive/'My Drive'/'IR Project'/* .
# !unzip pythonquestions.zip

In [ ]:
# !ls

# Imports

In [3]:
import chardet
import pandas as pd
import numpy as np
import nltk
import string
import copy
import keyword
import re
import pickle
import os
import heapq
import codecs
import math
from nltk.corpus import stopwords
from bs4 import BeautifulSoup as bs
from scipy.spatial.distance import cosine
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# !python -m nltk.downloader all > file.log

# Initialising necessary variables

In [4]:
stop_words = stopwords.words('english')
needed_words = ['what', 'which', 'if', 'while', 'for', 'between', 'into', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over','then','not','how','do']
removing_words = list(set(stop_words).difference(set(needed_words)))

In [5]:
lemmatizer = nltk.stem.WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

# Defining Functions

In [6]:
def preprocess(pd):
  pd = pd.str.lower()
  pd = pd.apply(lambda x: [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(x)])
  pd = pd.apply(lambda x: [item for item in x if item not in removing_words])
  pd = pd.apply(lambda x: [stemmer.stem(y) for y in x])
  pd = pd.str.join(' ')
  pd = pd.str.replace('[{}]'.format('$<>?@`\'"'), ' ')
  return pd

In [7]:
def extract_code(m):
  b = bs(m)
  t = b.find_all('code')
  for tag in t:
    tag.replace_with('')
  k = []
  for i in t:
    z = str(i).replace("<code>", "")
    z = str(z).replace("</code>", "")
    k.append(z)
  return k

In [8]:
def extract_body(m):
  b = bs(m)
  t = b.find_all('code')
  for tag in t:
      tag.replace_with('')
  a = list({tag.name for tag in b.find_all()})
  for i in a:
    b = str(b).replace("<"+str(i)+">", " ")
    b = str(b).replace("</"+str(i)+">", " ")
  return b

### Tagging Funcitons

In [9]:
def sort_list(list1, list2): 
    zipped_pairs = zip(list2, list1) 
    z = [x for _, x in sorted(zipped_pairs)] 
    return z 

In [10]:
def find_tags(tags,number):
    tags.sort()
    tags.reverse()
    sum1=0
    for i in range(0,number):
        sum1+=tags[i]
    avg=sum1/float(number)
    i=0
    counter=0
    while avg<tags[i]:
        counter+=1
        i+=1
    return counter

In [11]:
def magnitude(array_list):
    s=0
    for i in range(len(array_list)):
        s=s+(array_list[i]**2)
    return np.power(s,0.5)

In [12]:
def cosine_vec(query_vector,vec2):
    query_vector=np.array(query_vector)
    vec2=np.array(vec2)
    q1=magnitude(list(query_vector))
    cosine_similarity_val=np.dot(query_vector,vec2)/float(q1*magnitude(list(vec2)))

### Code Matching

In [13]:
def determineWordType(t1, t2):
    keylist = keyword.kwlist
    keys1 = 0
    keys2 = 0
    cf1 = 0
    cf2 = 0
    vars1 = 0
    vars2 = 0
    i = 0
    while i < len(t1):
        if t1[i] in keylist:
            keys1 += 1
        elif re.search(r':$', t1[i]):
            cf1 += 1
        else:
            vars1 += 1
        i += 1
    i = 0
    while i < len(t2):
        if t2[i] in keylist:
            keys2 += 1
        elif re.search(r':$', t2[i]):
            cf2 += 1
        else:
            vars2 += 1
        i += 1
    return [keys1, cf1, vars1], [keys2, cf2, vars2]

In [14]:
def matchCode(a, b):
    simscore = 0
    l1 = a.splitlines()
    l2 = b.splitlines()
    i = 0
    m = len(l1)
    n = len(l2)
    while i < min(m,n):
        t1=l1[i].split(" ")
        t2=l2[i].split(" ")
        vec1, vec2 = determineWordType(t1, t2)
        difvec = np.subtract(vec1, vec2)
        if sum(abs(difvec)) < min(len(t1), len(t2)):
            simscore += 1
        i += 1
    if min(m,n)==0:
      return 0.0
    return simscore/min(m,n)

In [15]:
def get_max(a):
  if len(a) == 0:
    return 0
  return max(a)

In [16]:
def cosine(a, b):
  m = a.multiply(b)
  numerator = m.sum(axis=1)
  det_a = np.sqrt(a.sum(axis = 1))
  det_b = np.sqrt(b.sum(axis = 1))
  return numerator / (det_a * det_b)

In [17]:
def code_sim(a, b):
    scores = []
    for i in a:
        scores.append(matchCode(i, b))

    for i in range(len(a)):
        scores[i] += matchSlidingWindow(i, b)
    return scores/2

In [18]:
def matchSlidingWindow(a, b):
    simscores = []
    l1 = a.splitlines()
    l2 = b.splitlines()
    m = len(l1)
    n = len(l2)
    wsize = min(m, n)
    i = 0
    if m < n:
        i += m
        k = 0
        while i < n:
            simscores.append(matchCode(''.join(l1), ''.join(l2[k:i])))
            k += 1
            i += 1      
    else:
        i += n
        k = 0
        while i < m:
            simscores.append(matchCode(''.join(l1[k:i]), ''.join(l2)))
            k += 1
            i += 1
            
    if len(simscores)==0:
      return 0.0
    return sum(simscores)/len(simscores)

# Tags

In [19]:
df = pd.read_csv("SampleQs.csv", encoding = "ISO-8859-1", index_col=0)
df.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,636977,76967.0,2009-03-12T00:48:24Z,0,Best way to remove duplicate characters (words...,<p>What would be the best way of removing any ...
1,13184947,1792223.0,2012-11-01T20:26:43Z,0,Python - Receipt-like output,<p>Im doing some homework and im breaking my h...
2,38190552,6000892.0,2016-07-04T18:34:10Z,0,Recovering Python file that is deleted yet sti...,<p>The other night I was running a Python scri...
3,26719088,281530.0,2014-11-03T16:59:11Z,3,Django 1.7 blank CharField/TextField convention,"<p>Using Django's new migration framework, let..."
4,10302672,786125.0,2012-04-24T17:09:41Z,3,Kivy Play Audio Not Working,<p>I'm attempting to run Kivy's audio example ...


In [20]:
df1 = pd.read_csv("SampleTs.csv", encoding = "ISO-8859-1", index_col=0)
print(df1.head())

         Id                Tag
0  10614000  google-app-engine
1  10614000      gae-datastore
2  10614000         python-2.7
3  10614000          datastore
4    636977             python


In [21]:
processed_q = copy.deepcopy(df)
processed_q['Title'] = preprocess(processed_q['Title'])
processed_q['Code'] = processed_q['Body'].apply(lambda x: extract_code(x))
processed_q['Body'] = preprocess(processed_q['Body'].apply(lambda x: extract_body(x)))
processed_q.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body,Code
0,636977,76967.0,2009-03-12T00:48:24Z,0,best way remov duplic charact (words) in string,what would best way remov duplic charact set c...,"[foo = 'h k k h2 h'\n, foo = 'h k h2' # order ..."
1,13184947,1792223.0,2012-11-01T20:26:43Z,0,python - receipt-lik output,im homework im break head around this.. either...,"[price = input(""Enter Price "")\ncash = input(""..."
2,38190552,6000892.0,2016-07-04T18:34:10Z,0,recov python file delet yet still visibl,night wa run python script wa calcul simul whi...,"[DMEmodel.py, http://DMEmodel.py, /home/user1,..."
3,26719088,281530.0,2014-11-03T16:59:11Z,3,django 1.7 blank charfield/textfield convent,"use django new migrat framework, let say fol...",[class TestModel(models.Model):\n field_1 =...
4,10302672,786125.0,2012-04-24T17:09:41Z,3,kivi play audio not work,i m attempt run kivy audio exampl found a hr...,[sound = SoundLoader.load(filename='test.wav')...


In [22]:
id3=processed_q['Id']
body=processed_q['Body']
title=processed_q['Title']
code=processed_q['Code']
id3=list(id3)
body=list(body)
title=list(title)
code=list(code)

In [23]:
questions_with_hash={}
for i in range(len(id3)):
    temp={}
    temp['body']=body[i]
    temp['title']=title[i]
    temp['code']=code[i]
    questions_with_hash[id3[i]]=temp

In [2]:
s=[1,2,3]
s.pop(0)
print(s)

[2, 3]


In [6]:
l1=0
l2=9
a=[l1,l2]
max(a)

9

In [9]:
np.matmul(np.transpose([[1,2],[5,4]]),np.array([[1,5],[9,8]]))

array([[46, 45],
       [38, 42]])

In [12]:
import networkx as nx
def steady_sim_neigh(A,B):
    G1=nx.from_numpy_matrix(A)
    G2=nx.from_numpy_matrix(B)
    
    w1, v1 = LA.eig(np.matmul(np.transpose(A),np.array(A)))
    v1=v1.transpose()
    sim1=np.array(np.matmul(np.matrix(A),np.matrix(v1)))
    w1, v1 = LA.eig(np.matmul(np.transpose(B),np.array(B)))
    v1=v1.transpose()
    sim2=np.array(np.matmul(np.matrix(B),np.matrix(v1)))
    a1=LA.norm(sim1)
    a2=LA.norm(sim2)
    a=a1+a2
    
    w1, v1 = LA.eig(np.matmul(np.array(A),np.transpose(A)))
    v1=v1.transpose()
    sim1=np.array(np.matmul(np.matrix(A),np.matrix(v1)))
    w1, v1 = LA.eig(np.matmul(np.array(B),np.transpose(B)))
    v1=v1.transpose()
    sim2=np.array(np.matmul(np.matrix(B),np.matrix(v1)))
    b1=LA.norm(sim1)
    b2=LA.norm(sim2)
    b=b1+b2
    return (0.5*np.absolute(a1-a2)/float(a)+0.5*np.absolute(b1-b2)/float(b))

In [16]:
from numpy import linalg as LA
steady_sim_neigh(np.array([[1,2],[5,6]]),np.array([[1,2],[5,6]]))

0.0

In [24]:
id1=df1['Id']
id1=list(id1)
tag=df1['Tag']
tag=list(tag)

In [25]:
quesid_invertedindex={}
for i in range(len(id1)):
    if tag[i] not in quesid_invertedindex.keys():
        temp=[]
        temp.append(id1[i])
        quesid_invertedindex[tag[i]]=temp
    else:
        quesid_invertedindex[tag[i]].append(id1[i])

In [26]:
flag={}
for i in quesid_invertedindex.keys():
    flag[i]=0

In [27]:
tag_list={}
counter=0
for i in range(len(tag)):
    counter+=1
    if tag[i]=='python' or tag[i]=='python-2.7' or tag[i]=='python-3.0':
        continue
    if counter%50000==0:
        print("Counter : ",counter)
    try:
        flag2=tag_list[tag[i]]
        temp_t=questions_with_hash[id1[i]]     
        tag_list[tag[i]]['body']+=temp_t['body']
        tag_list[tag[i]]['title']+=temp_t['title']
        tag_list[tag[i]]['code'].append(temp_t['code'])
        flag[tag[i]]+=1                
    except:
        temp_t=questions_with_hash[id1[i]]
        temp={}
        temp['body']=temp_t['body']
        temp['title']=temp_t['title']
        temp1=[]
        temp1.append(temp_t['code'])
        temp['code']=temp1
        tag_list[tag[i]]=temp
        flag[tag[i]]+=1

Counter :  50000
Counter :  100000
Counter :  150000
Counter :  200000
Counter :  250000
Counter :  300000
Counter :  350000
Counter :  500000


In [28]:
body_corpus = pickle.load(open("body.txt", "rb") )
title_corpus = pickle.load(open("title.txt", "rb" ) )

In [29]:

title_corpus_main=[]
title_corpus_main.append(title_corpus)

body_corpus_main=[]
body_corpus_main.append(body_corpus)#For Title using question body

vc_title_tag = CountVectorizer() #TfidfVectorizer()
vec_title = vc_title_tag.fit_transform(title_corpus_main)

In [30]:
vc_body_tag = CountVectorizer() #TfidfVectorizer()
vec_body = vc_body_tag.fit_transform(body_corpus_main)

In [50]:
def tag_detector(query_body, query_code, weight_title, weight_body, weight_code, rel):

  q_title = vc_title_tag.transform(preprocess(pd.DataFrame([query_body])[0]))
  q_title1 = vc_body_tag.transform(preprocess(pd.DataFrame([query_body])[0]))
  
  similarity_body=[]
  similarity_title=[]
  similarity_code=[]
  tag_similar=[]
  counter=0
  for i in tag_list.keys():
      counter+=1
      if counter%2000==0:
          print("Counter : ",counter)
      a=[]
      a.append(tag_list[i]['body'])
      sim1=vc_body_tag.transform(a)
      similarity_body.append(cosine_similarity(sim1, q_title1)[0][0])
      a=[]
      a.append(tag_list[i]['title'])
      sim1=vc_title_tag.transform(a)
      similarity_title.append(cosine_similarity(sim1, q_title)[0][0])    
      tag_similar.append(i)
#         scores=[]
#         for j in tag_list[i]['code']:
#             scores.append(code_sim(j[0],query_code))
#         similarity_code.append(max(scores))

  main_score=[]
  for i in range(len(similarity_body)):
      main_score.append(similarity_body[i]*weight_body+similarity_title[i]*weight_title)
  a=copy.deepcopy(np.array(main_score))
  
  rel_opt=find_tags(list(a),rel)

  #Another approach to take top relevant tags
  list_top=heapq.nlargest(rel_opt, range(len(a)), a.take)
  
  tags1=[]
  for i in range(len(list_top)):
      tags1.append(tag_similar[list_top[i]])
      print(tag_similar[list_top[i]])

  ques_id=[]
  for i in range(len(tags1)):
      ques_id+=quesid_invertedindex[tags1[i]]
  
  return ques_id

# Questions

In [59]:
def filter_questions(tagged_question_ids, question, code, alpha, beta, gamma, top_count):
    a = pd.DataFrame(tagged_question_ids).rename({0:'Id'}, axis = 1)

    processed_tag = pd.merge(a, processed_q, how='inner', on="Id")
    processed_tag = processed_tag.drop_duplicates('Id')
    
    VC_title = CountVectorizer() #TfidfVectorizer()
    X_title = VC_title.fit_transform(processed_tag['Title'])
    
    VC_body = CountVectorizer() #TfidfVectorizer()
    X_body = VC_body.fit_transform(processed_tag['Body'])

    Q_title = VC_title.transform(preprocess(pd.DataFrame([question])[0]))
    vec_title = cosine_similarity(X_title, Q_title)[0][0]

    Q_body = VC_body.transform(preprocess(pd.DataFrame([question])[0]))
    vec_body = cosine_similarity(X_body, Q_body)[0][0]

#     vec_title = np.nan_to_num(vec_title)
#     vec_body = np.nan_to_num(vec_body)
    X_title = np.nan_to_num(X_title)
    X_body = np.nan_to_num(X_body)

    processed_tag['Code_sim'] = processed_tag['Code'].apply(lambda x: [matchCode(code, i) for i in x])
    processed_tag['Code_sim_max'] = processed_tag['Code_sim'].apply(lambda x: get_max(x))

    processed_tag.head()

    vec = alpha * vec_title + gamma * vec_body

    top_questions = np.array(vec).flatten().argsort()[-top_count:][::-1]

    top_question_id = []
    for i in top_questions:
        top_question_id.append(processed_q['Id'][i])

    top_questions_pd = pd.DataFrame(top_question_id).rename({0:'ParentId'}, axis=1)
    return top_questions_pd

# Filtering Answers

In [39]:
answers = pd.read_csv('SampleAs.csv', encoding = "ISO-8859-1", index_col=0)

In [70]:
def filter_answers(question, top_questions_pd, code, alpha, top_count):
    processed_a = pd.merge(top_questions_pd, answers, how='inner')
    processed_a['Code'] = processed_a['Body'].apply(lambda x: extract_code(x))
    processed_a['Body'] = preprocess(processed_a['Body'].apply(lambda x: extract_body(x)))

    VC_body_ans = CountVectorizer() #TfidfVectorizer()
    X_body_ans = VC_body_ans.fit_transform(processed_a['Body'])

    Q_body_ans = VC_body_ans.transform(preprocess(pd.DataFrame([question])[0]))
    vec_body_ans = cosine_similarity(X_body_ans, Q_body_ans)[0][0]

#     vec_body_ans = np.nan_to_num(vec_body_ans)

    processed_a['Code_sim'] = processed_a['Code'].apply(lambda x: [matchCode(code, i) for i in x])
    processed_a['Code_sim_max'] = processed_a['Code_sim'].apply(lambda x: get_max(x))

    vec_ans = (alpha * vec_body_ans)

    top_answers = np.array(vec_ans).flatten().argsort()[-top_count:][::-1]

    top_answer_id = []
    for i in top_answers:
        top_answer_id.append(processed_a['Id'][i])
    top_answers_id = pd.DataFrame(top_answer_id).rename({0:'Id'}, axis=1)
    return pd.merge(top_answers_id, processed_a, how='inner')

In [41]:
question

'Python code file numpy pandas not working'

# Search

In [42]:
question = "Python code file numpy pandas not working"
code = "import csv as csv\nimport numpy as np\ncsv_file_object = csv.reader(open('train.csv', 'rb'))\nheader = csv_file_object.next()\ndata = []\nfor row in csv_file_object:"


In [43]:
print(code)

import csv as csv
import numpy as np
csv_file_object = csv.reader(open('train.csv', 'rb'))
header = csv_file_object.next()
data = []
for row in csv_file_object:


In [51]:
tagged_question_ids = tag_detector(question, code, 0.6, 0.4, 0, 100)

Counter :  2000
Counter :  4000
Counter :  6000
Counter :  8000
Counter :  10000
last.fm
file-transfer
code-generation
yad
vscode
bytecode-manipulation
weka
yui-compressor
file-io
sfml
file
system
geektool
filesystems
io
zipfile
notin
exe
flac
envoy
compatibility
type-library
ess
file-processing
python-3.2
temporary-files
batch-file
bytecode
2to3
decompression
xz
python-embedding
compression
zip
filepath
windows-server-2003
file-writing
jar
c++
filereader
python-mode


In [52]:
len(tagged_question_ids)

4572

In [60]:
top_questions_pd = filter_questions(tagged_question_ids, question, code, 0.6, 0, 0.4, 2)

In [61]:
top_questions_pd.head()

,ParentId
0,636977


In [62]:
top_questions_pd.shape

(1, 1)

In [77]:
top_answers = filter_answers(question, top_questions_pd, code, 0.4, 100)

In [78]:
# max_index = []
# for i in range(top_answers:
#     print(i)
#     max_index.append(np.argmax(i['Code_sim']))

In [79]:
top_answers

,Id,ParentId,OwnerUserId,CreationDate,Score,Body,Code,Code_sim,Code_sim_max
0,636982,636977,10661.0,2009-03-12T00:49:59Z,8,do mean that uniqu space-delimit word in par...,[' '.join( set( someString.split() ) )\n],[0.0],0.0
